In [1]:
#GW
!pip install tf-keras==2.19
#GW
!pip install transformers datasets keras-nlp 
!pip install keras>=3 tensorflow-text huggingface-hub peft langchain_community chromadb sentence-transformers llama-index faiss-cpu #GW
#!pip install keras tensorflow-text huggingface-hub peft langchain_community chromadb sentence-transformers llama-index faiss-cpu
!pip install python-docx PyMuPDF gradio python-pptx  pypdf unstructured pymupdf
!pip install langchain langchain_core langchain-google-vertexai

  Using cached tensorflow-2.19.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
Using cached tensorflow-2.19.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (644.8 MB)
Using cached tensorboard-2.19.0-py3-none-any.whl (5.5 MB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.1
    Uninstalling tensorflow-2.18.1:
      Successfully uninstalled tensorflow-2.18.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.19.0 which is incompatible.
  Using cached tensorflow-2.1

In [2]:
%pip install --upgrade --quiet kagglehub

Note: you may need to restart the kernel to use updated packages.


In [1]:

import os
import keras
import keras_nlp
import keras_hub
from IPython.display import Markdown
import textwrap
import gradio as gr
import langchain_core
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, UnstructuredWordDocumentLoader, UnstructuredPowerPointLoader
from langchain.schema.runnable import Runnable
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import pymupdf
import fitz 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from docx import Document
from pptx import Presentation
from langchain.schema import Document


2025-05-25 09:53:10.869171: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748166790.916044   29712 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748166790.928986   29712 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-25 09:53:11.029025: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#configuring notebook
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [3]:
#defining fn to display prompt and response prettily

def display_chat(prompt, response):
  '''Displays an LLM prompt and response in a pretty way.'''
  prompt = prompt.replace('\n\n','')
  prompt = prompt.replace('\n','')
  formatted_prompt = "🙋‍♂️" + prompt + ""
  response = response.replace('•', '  *')
  response = textwrap.indent(response, '', predicate=lambda _: True)
  response = response.replace('\n\n','')
  response = response.replace('\n','')
  response = response.replace("```","")
  formatted_text = "🤖" + response + ""
  return Markdown(formatted_prompt+formatted_text)
     

In [4]:
import kagglehub

kagglehub.login()
#38c08016f7eb7512a1352518ea800966

In [6]:
#load model of choice

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")
gemma_lm.summary()

2025-05-25 09:54:00.658338: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-05-25 09:54:00.696721: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2359296000 exceeds 10% of free system memory.
2025-05-25 09:54:04.994529: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2359296000 exceeds 10% of free system memory.
2025-05-25 09:54:05.682077: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2359296000 exceeds 10% of free system memory.
2025-05-25 09:54:29.194172: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2359296000 exceeds 10% of free system memory.
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

load documents --> chunk --> embedding into vector space--> indexing vector space--> query + retrieval from a vector space --> top p matches or contexts + LLM --> prompt template --> response

In [7]:
#function for loading documents
from langchain.document_loaders import DirectoryLoader, PyPDFLoader, UnstructuredWordDocumentLoader, UnstructuredPowerPointLoader

# Load PDFs
print("Load pdfs..")
#pdf_loader = DirectoryLoader("./././RAG_pdf", glob="**/*.pdf", loader_cls=PyPDFLoader)
pdf_loader = DirectoryLoader("./././RAG_pdf", glob="**/Contact.pdf", loader_cls=PyPDFLoader)
pdf_docs = pdf_loader.load()

# Add metadata for PDFs which is optional
print("Load metadata...")
for doc in pdf_docs:
    doc.metadata = {"source": "RAG_pdf", "file_name": doc.metadata["source"]}

# Load DOCX
print("Load docx...")
docx_loader = DirectoryLoader('./././RAG_docx', glob="**/*.docx", loader_cls=UnstructuredWordDocumentLoader)
docx_docs = docx_loader.load()

# Add metadata for DOCX
print("Load docx_docs...")
for doc in docx_docs:
    doc.metadata = {"source": "RAG_docx", "file_name": doc.metadata["source"]}

# Load PPTX
print("loading pptx...")
#GW pptx_loader = DirectoryLoader('./././RAG_pptx', glob="**/*.pptx", loader_cls=UnstructuredPowerPointLoader)
pptx_loader = DirectoryLoader('./././RAG_pptx', glob="**/ISMP*.pptx", loader_cls=UnstructuredPowerPointLoader)
pptx_docs = pptx_loader.load()


# Add metadata for PPTX
for doc in pptx_docs:
    doc.metadata = {"source": "RAG_pptx", "file_name": doc.metadata["source"]}


# Combine all documents
all_docs = pdf_docs + docx_docs + pptx_docs

Load pdfs..
Load metadata...
Load docx...
Load docx_docs...
loading pptx...


In [8]:

os.listdir('/teamspace/studios/this_studio/RAG_docx')

['Advising Services _ Portland State University.docx',
 'FAQ for International Student Mentors 2024.docx',
 'OISSS and Immigration Scenarios.docx',
 'On Campus Childcare _ Portland State University.docx']

In [9]:
# For LangChain documents (metadata)
#from langchain.docstore.document import Document as LangDocument

# For Word document processing
from docx import Document as DocxDocument

# Function to create documents with metadata (source folder). this is optional
def create_documents_with_metadata(texts, source_folder):
    documents = []
    for text in texts:
        doc = Document(page_content=text, metadata={"source_folder": source_folder})
        documents.append(doc)
    return documents


In [10]:
# alt functions for Loading document content from a PDF
def load_pdfs(folder_path):
    texts = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".pdf"):
            file_path = os.path.join(folder_path, file_name)
            pdf = fitz.open(file_path) 
            
            # Extract text from each page
            for page_num in range(len(pdf)):
                page = pdf.load_page(page_num)
                text = page.get_text()
                texts.append(text)
            
            pdf.close()  # Close after processing each file
    return texts
        
    

def load_docx(folder_path):
    texts = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".docx"):
            file_path = os.path.join(folder_path, file_name)
            doc = DocxDocument(file_path)  # Open the Word document
            
            # Extract text from each paragraph
            for paragraph in doc.paragraphs:
                if paragraph.text.strip():
                    texts.append(paragraph.text)
    return texts



def load_pptx(folder_path):
    texts = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".pptx"):
            file_path = os.path.join(folder_path, file_name)
            presentation = Presentation(file_path)  # Open the PPT file
            
            # Extract text from each slide
            for slide in presentation.slides:
                for shape in slide.shapes:
                    if hasattr(shape, "text"):
                        texts.append(shape.text)
    return texts


def load_all(folder_paths):
    pdf_texts = load_pdfs(folder_paths['pdf'])
    docx_texts = load_docx(folder_paths['docx'])
    pptx_texts = load_pptx(folder_paths['pptx'])


     #Create documents with metadata (source folder) for each type of file
    pdf_docs = create_documents_with_metadata(pdf_texts, "pdf")
    docx_docs = create_documents_with_metadata(docx_texts, "docx")
    pptx_docs = create_documents_with_metadata(pptx_texts, "pptx")


    # Combine all text into one list
    all_texts = pdf_texts + docx_texts + pptx_texts
    return all_texts



folder_paths = {
    'pdf': './././RAG_pdf',
    'docx': './././RAG_docx',
    'pptx': './././RAG_pptx'
}

all_texts = load_all(folder_paths)
print(f"Loaded {len(all_texts)} documents with metadata")



Loaded 970 documents with metadata


In [11]:
def clean_text(text):
    text = text.replace("\n", " ").strip()  # Remove newlines
    text = " ".join(text.split())  # Remove extra spaces
    return text

for doc in all_docs:

    doc.page_content = clean_text(doc.page_content)
    #print(doc.page_content[-1]) 

In [12]:
#alt fn
'''
def clean_text(text):
    text = text.replace("\n", " ").strip()  # Remove newlines
    text = " ".join(text.split())  # Remove extra spaces
    return text

# Clean all items in all_texts list
all_texts = [clean_text(text) for text in all_texts]
'''

#making this compatible to langchain's pipeline, so we can add metadata e.g source, filename etc easily

source_docs = [Document(page_content=text) for text in all_texts] #we dont need this line with the create_documents_with_metadata fn above 

# Clean text content within Document objects
def clean_text(text):
    text = text.replace("\n", " ").strip()  # Remove newlines
    text = " ".join(text.split())  # Remove extra spaces
    return text

# Clean each document
for sdoc in source_docs:
    sdoc.page_content = clean_text(sdoc.page_content)


In [13]:
#Split Documents into Chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Adjust based on your needs
    chunk_overlap=250
)
chunks = text_splitter.split_documents(all_docs)#put source_docs

# Print out chunked documents (optional for checking)
# for chunk in chunks:
    #print(chunk.page_content)


'''
from langchain.text_splitter import CharacterTextSplitter

# Create the text splitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# Split the documents
split_docs = []
for doc in all_docs:
    split_docs.extend(text_splitter.split_documents([doc]))

# Now, split_docs contains chunks of text from the original documents

'''

'\nfrom langchain.text_splitter import CharacterTextSplitter\n\n# Create the text splitter\ntext_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)\n\n# Split the documents\nsplit_docs = []\nfor doc in all_docs:\n    split_docs.extend(text_splitter.split_documents([doc]))\n\n# Now, split_docs contains chunks of text from the original documents\n\n'

In [ ]:
'''#will troubleshoot later if i ever want to use hub.load...
! pip install tensorflow==2.15.0 #GW
! pip install tensorflow-hub 

#GW
import tensorflow as tf
#import tensorflow_hub as hub
#GW import tensorflow as tf

print("TensorFlow version:", tf.__version__)  # Should be 2.15.0
#print("TensorFlow Hub version:", hub.__version__)  # Should be 0.16.1
'''

In [14]:
#GW
! pip install chromadb

In [ ]:
#GW 
''' to trblsht 
import tensorflow_hub as hub
#GW

model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
'''

#GW
'''
def embed_fn(texts):
    return model(texts).numpy().tolist()

# Create a Chroma vector database from the documents
# Important: Make sure to delete previous db (if any) or else retrieval returns lots of duplicates :)
try:
  db.delete_collection()
except:
  pass
db = Chroma.from_documents(chunks, embed_fn)
'''

In [60]:
#GW from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
#create embedding from documents 

# Load the embeddings model
# NOTE: You might need to experiment with different models
embeddings = HuggingFaceEmbeddings(model_name= 'all-MiniLM-L6-v2') #GW SentenceTransformer('all-MiniLM-L6-v2'))

# Create a Chroma vector database from the documents
# Important: Make sure to delete previous db (if any) or else retrieval returns lots of duplicates :)
try:
  db.delete_collection()
except:
  pass
# Create Chroma vector store from chunks and save vectore store too
db = Chroma.from_documents(chunks, embeddings, persist_directory="./chroma_db")
db.persist()  # Save for later reuse

#to load from saved vector store
# Load Chroma index
#chroma_store = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)




In [83]:
# Create a retriever from the vector database
# NOTE: You might need to experiment with retrieval parameters
#retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 2, "score_threshold": 0.3})
retriever = db.as_retriever(search_type="similarity",search_kwargs={"k": 3} ) # Return top 5 results)
'''
#the chroma is a better option for our use case
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Initialize the OpenAI Embeddings
embeddings = OpenAIEmbeddings()

# Store embeddings in FAISS directly from split_docs
vector_store = FAISS.from_documents(chunks, embeddings)
vector_store.save_local("faiss_index")  # Save for later reuse
# to Load FAISS index from the vector store
faiss_store = FAISS.load_local("faiss_index", embeddings)
'''



'\n#the chroma is a better option for our use case\nfrom langchain.embeddings import OpenAIEmbeddings\nfrom langchain.vectorstores import FAISS\n\n# Initialize the OpenAI Embeddings\nembeddings = OpenAIEmbeddings()\n\n# Store embeddings in FAISS directly from split_docs\nvector_store = FAISS.from_documents(chunks, embeddings)\nvector_store.save_local("faiss_index")  # Save for later reuse\n# to Load FAISS index from the vector store\nfaiss_store = FAISS.load_local("faiss_index", embeddings)\n'

In [44]:
'''
#optionally, we can add a reranker, esp when wer are working with multistep reasoning, and in this case increase k to retrieve more chunks

from sentence_transformers import CrossEncoder

# Load a cross-encoder model for reranking
reranker_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

# Input: Query and list of documents
query = "Your query"
documents = ["Doc 1", "Doc 2", "Doc 3"]

# Score each document
scores = reranker_model.predict([(query, doc) for doc in documents])

# Sort documents by score
ranked_documents = [doc for _, doc in sorted(zip(scores, documents), reverse=True)]
'''

'\n#optionally, we can add a reranker, esp when wer are working with multistep reasoning, and in this case increase k to retrieve more chunks\n\nfrom sentence_transformers import CrossEncoder\n\n# Load a cross-encoder model for reranking\nreranker_model = CrossEncoder(\'cross-encoder/ms-marco-MiniLM-L-12-v2\')\n\n# Input: Query and list of documents\nquery = "Your query"\ndocuments = ["Doc 1", "Doc 2", "Doc 3"]\n\n# Score each document\nscores = reranker_model.predict([(query, doc) for doc in documents])\n\n# Sort documents by score\nranked_documents = [doc for _, doc in sorted(zip(scores, documents), reverse=True)]\n'

In [84]:
#to retain memory of chat...this is optional as well.turn-based chat management.

class ChatState():
  """
  Manages the conversation history for a turn-based chatbot
  Follows the turn-based conversation guidelines for the Gemma family of models
  documented at https://ai.google.dev/gemma/docs/formatting
  """

  __START_TURN_USER__ = "user\n"
  __START_TURN_MODEL__ = "model\n"
  __END_TURN__ = "\n"

  def __init__(self, model, system=""):
    """
    Initializes the chat state.

    Args:
        model: The language model to use for generating responses.
        system: (Optional) System instructions or bot description.
    """
    self.model = model
    self.system = system
    self.history = []

  def add_to_history_as_user(self, message):
      """
      Adds a user message to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_USER__ + message + self.__END_TURN__)

  def add_to_history_as_model(self, message):
      """
      Adds a model response to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_MODEL__ + message + self.__END_TURN__)

  def get_history(self):
      """
      Returns the entire chat history as a single string.
      """
      return "".join([*self.history])

  def get_full_prompt(self):
    """
    Builds the prompt for the language model, including history and system description.
    """
    prompt = self.get_history() + self.__START_TURN_MODEL__
    if len(self.system)>0:
      prompt = self.system + "\n" + prompt
    return prompt

  def send_message(self, message):
    """
    Handles sending a user message and getting a model response.

    Args:
        message: The user's message.

    Returns:
        The model's response.
    """
    self.add_to_history_as_user(message)
    prompt = self.get_full_prompt()
    response = self.model.generate(prompt, max_length=1024)
    result = response.replace(prompt, "")  # Extract only the new response
    self.add_to_history_as_model(result)
    return result

In [85]:
#testing the rtriever on prompts. we use the list of returned documents as context

ret_docs = retriever.invoke("how do i book appointment with an advisor")
print("retreived", len(ret_docs), "documents")
print(ret_docs)


query = "how do i book appointment with an advisor"
results = retriever.invoke(query)

for i, result in enumerate(results):
    print(f"Result {i + 1}:\n{result.page_content}\n")


retreived 3 documents
[Document(metadata={'source': 'RAG_docx', 'file_name': 'RAG_docx/Advising Services _ Portland State University.docx'}, page_content='email intl-sponsor@pdx.edu. You can meet with any International Student advisor for questions about your F-1 or J-1 status. Visiting Scholars: Visiting scholars, faculty, and researchers on the J-1 visa should make an appointment with David Brandt. New and prospective students: If you have questions about applying to PSU, transcripts, the English language requirement, or other admissions questions, contact the International Admissions office. 2/9 MAKE AN APPOINTMENT To see all available appointment times (not advisor specific): 1. Follow the link for International Student Advisors or Student Life Advisors 2. On the advisor list page, scroll to the bottom to choose "No Preference." 3. Select a time from the calendar. 4. Enter your details and reserve the appointment. Meet an International Student Advisor: https://immigration-advisors.

In [64]:
#create a prompt template

from langchain.prompts import PromptTemplate
# Create a prompt template for RAG
template = """You are a helpful assistant. Use the following context to answer the question accurately.
Context:
{context}

Question: 
{question}

Answer:
"""
prompt_template = PromptTemplate(template=template, input_variables=["context", "question"])

In [33]:
!pip show torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: torch
Version: 2.2.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /system/conda/miniconda3/envs/cloudspace/lib/python3.10/site-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, lightning, litdata, peft, pytorch-lightning, sentence-transformers, torchmetrics, torchvision


In [72]:
#create the RAG  RetrievalQA chain using huggong face automodel and pipeline. we could use the kerasnlp model already downloaded above
from transformers import pipeline, GenerationConfig, AutoTokenizer,  AutoModelForSeq2SeqLM
from langchain.llms import HuggingFacePipeline


# Load the FLAN-T5-Large model and tokenizer
model_name = "google/flan-t5-large" # we can use larger model to avoid chunking or truncation during genration
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")
text_generator = pipeline(
    "text2text-generation",  # Use "text2text-generation" for flexibility
    model=model,
    tokenizer=tokenizer,
    max_length=512,  # Maximum length of the generated text,
    min_length=30,   # Minimum length of the generated text
    do_sample=True,  # Enable sampling for diverse outputs
    temperature=0.7, # Control randomness (lower = more deterministic)
    top_k=50,        # Limit sampling to the top-k tokens
    top_p=0.95       # Nucleus sampling (cumulative probability threshold)
)

# Wrap with LangChain
llm = HuggingFacePipeline(pipeline=text_generator)


'''

or use

model_name = "google/flan-t5-large"

 or use 

 
from langchain.llms import HuggingFacePipeline
import torch

model_name = "meta-llama/Llama-2-7b-chat-hf"
llm = HuggingFacePipeline.from_model_id(
    model_id=model_name,
    task="text-generation",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16}
)

#or say

from langchain.chat_models import ChatGoogleGenerativeAI

# Initialize Gemini model
llm = ChatGoogleGenerativeAI(model="gemini-pro") 

from langchain_google_vertexai import VertexAI  # <button class="citation-flag" data-index="7">

llm = VertexAI(model_name="text-bison@001")  # Or another Vertex AI model
'''

Some parameters are on the meta device because they were offloaded to the disk and cpu.
Device set to use cpu


'\n\nor use\n\nmodel_name = "google/flan-t5-large"\n\n or use \n\n \nfrom langchain.llms import HuggingFacePipeline\nimport torch\n\nmodel_name = "meta-llama/Llama-2-7b-chat-hf"\nllm = HuggingFacePipeline.from_model_id(\n    model_id=model_name,\n    task="text-generation",\n    device_map="auto",\n    model_kwargs={"torch_dtype": torch.float16}\n)\n\n#or say\n\nfrom langchain.chat_models import ChatGoogleGenerativeAI\n\n# Initialize Gemini model\nllm = ChatGoogleGenerativeAI(model="gemini-pro") \n\nfrom langchain_google_vertexai import VertexAI  # <button class="citation-flag" data-index="7">\n\nllm = VertexAI(model_name="text-bison@001")  # Or another Vertex AI model\n'

In [35]:
# Initialize ChatState if in use
chat_state = ChatState(model=llm, system="You are an expert assistant.")

In [86]:

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever, #reranker
    input_key="query",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_template}
)

In [67]:
query = "How do I book an appointment with an advisor?"

if not query.strip():
    raise ValueError("Query cannot be empty.")

In [68]:
retrieved_documents = retriever.get_relevant_documents(query)

if not retrieved_documents:
    print("No relevant documents found for the query.")
else:
    print(f"Found {len(retrieved_documents)} relevant documents.")

Found 5 relevant documents.


In [69]:
# Example query
query = "How do I book an appointment with an advisor?"

# Tokenize the query
tokenized_output = tokenizer(query, return_tensors="pt")  # "pt" for PyTorch tensors

# Print the tokenized output
print("Tokenized Output:")
print(tokenized_output)

# Decode the token IDs back to text
decoded_text = tokenizer.decode(tokenized_output["input_ids"][0], skip_special_tokens=True)
print("\nDecoded Text:")
print(decoded_text)

Tokenized Output:
{'input_ids': tensor([[ 571,  103,   27,  484,   46, 4141,   28,   46, 8815,   58,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

Decoded Text:
How do I book an appointment with an advisor?


In [70]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings = embedding_model.embed_query(query)

print(f"Embedding size: {len(embeddings)}")

Embedding size: 384


In [88]:
query = "how do i book appointment with an advisor?"
result = qa_chain({"query": query})

# Output the answer
print("\nAnswer:\n", result["result"])

# (Optional) Show which chunks were used
print("\nSources:")
for doc in result["source_documents"]:
    print(f"- {doc.page_content[:200]}...")


'''
#we can also try this

# Access the 'result' key directly
retrieved_text = result['result'] if 'result' in result else "No context available."

# Create the prompt using the retrieved text
prompt = f"Use the following information to answer the question:\n{retrieved_text}\nQuestion: {query}\nAnswer:"

# Generate the response using the prompt
response = generate_response(prompt)

# Display the question and answer
print("Question:", query)
print("Answer:", response)

'''

'''
if we have the error: Token indices sequence length is longer than the specified maximum sequence length for this model (1255 > 512). Running this sequence through the model will result in indexing errors

we can tokenize query with truncation:
example:
query = "How do I book an appointment with an advisor?"

# Tokenize with truncation
tokenized_output = tokenizer(
    query,
    return_tensors="pt",
    max_length=512,        # Truncate input to fit within the model's limit
    truncation=True        # Automatically truncate if input is too long
)

# Decode truncated input
truncated_query = tokenizer.decode(tokenized_output["input_ids"][0], skip_special_tokens=True)
print("\nTruncated Query:")
print(truncated_query)

# Generate response
response = text_generator(truncated_query)
print("\nGenerated Response:")
print(response[0]['generated_text'])
'''



Answer:
 1. Follow the link for International Student Advisors or Student Life Advisors 2. On the advisor list page, scroll to the bottom to choose "No Preference." 3. Select a time from the calendar 4. Enter your details and reserve the appointment.

Sources:
- email intl-sponsor@pdx.edu. You can meet with any International Student advisor for questions about your F-1 or J-1 status. Visiting Scholars: Visiting scholars, faculty, and researchers on the J-1 vi...
- International Students ADVISING SERVICES IKARL MILLER CENTER & State FRONT DESK SERVICES For general information and quick questions, visit our front desk services. A staff or student member can chat ...
- divide document review requests based on PSU ID so that requests can be processed consistently for all students. You can meet with any International Student Advisor whose availability fits your schedu...


'\nif we have the error: Token indices sequence length is longer than the specified maximum sequence length for this model (1255 > 512). Running this sequence through the model will result in indexing errors\n\nwe can tokenize query with truncation:\nexample:\nquery = "How do I book an appointment with an advisor?"\n\n# Tokenize with truncation\ntokenized_output = tokenizer(\n    query,\n    return_tensors="pt",\n    max_length=512,        # Truncate input to fit within the model\'s limit\n    truncation=True        # Automatically truncate if input is too long\n)\n\n# Decode truncated input\ntruncated_query = tokenizer.decode(tokenized_output["input_ids"][0], skip_special_tokens=True)\nprint("\nTruncated Query:")\nprint(truncated_query)\n\n# Generate response\nresponse = text_generator(truncated_query)\nprint("\nGenerated Response:")\nprint(response[0][\'generated_text\'])\n'

In [89]:
query = "what website can i go to book appointment with an advisor?"
result = qa_chain({"query": query})

# Output the answer
print("\nAnswer:\n", result["result"])

# (Optional) Show which chunks were used
print("\nSources:")
for doc in result["source_documents"]:
    print(f"- {doc.page_content[:200]}...")



Answer:
 https://immigration-advisors.youcanbook.me/ Meet a Student Life Advisor: https://student-life-advisors.youcanbook.me/

Sources:
- email intl-sponsor@pdx.edu. You can meet with any International Student advisor for questions about your F-1 or J-1 status. Visiting Scholars: Visiting scholars, faculty, and researchers on the J-1 vi...
- International Students ADVISING SERVICES IKARL MILLER CENTER & State FRONT DESK SERVICES For general information and quick questions, visit our front desk services. A staff or student member can chat ...
- divide document review requests based on PSU ID so that requests can be processed consistently for all students. You can meet with any International Student Advisor whose availability fits your schedu...


In [91]:
#if working with chat state, this helps maintain history

query = "where is KMC located?"

# Add user query to history
chat_state.add_to_history_as_user(query)

# Retrieve relevant context
retrieved_docs = retriever.get_relevant_documents(query)
context = "\n".join([doc.page_content for doc in retrieved_docs])

# Build full prompt with retrieved context
full_prompt = f"{context}\n\n{chat_state.get_full_prompt()}"

# Get model response
response = llm.invoke(full_prompt)
#or using kerasnlpmodel
#gemma_lm.generate(prompt, max_length=256)

# Store the model's response in the chat history
chat_state.add_to_history_as_model(response)

# Output the result
print("\nAnswer:\n", response)



Answer:
 You are an expert assistant. user where is kmc? model : f x (0) in 2/2 International Students ADVISING SERVICES IKARL MILLER CENTER & State FRONT DESK SERVICES For general information and quick questions, visit our front desk services. A staff or student member can chat with you and provide information, resources, and help schedule an appointment with an International Student Advisor. user Where is KMC? model


In [1]:
#instead of the cell above use below to avoid error in indexing due to long input and context due to 512 max capacity. see last cell

# If working with chat state, this helps maintain history
query = "Where is KMC located?"

# Add user query to history
chat_state.add_to_history_as_user(query)

# Retrieve relevant context
retrieved_docs = retriever.get_relevant_documents(query)

# Inspect retrieved documents
print("Retrieved Documents:")
for i, doc in enumerate(retrieved_docs):
    print(f"Document {i + 1}: {doc.page_content[:200]}...")
    
context = "\n".join([doc.page_content for doc in retrieved_docs])

# Build full prompt with retrieved context
full_prompt = f"{context}\n\n{chat_state.get_full_prompt()}"

# Tokenize and truncate the full prompt to fit within the model's token limit
max_input_length = 512  # Maximum input length for google/flan-t5-large
max_query_length = 128  # Reserve space for the query
max_context_length = max_input_length - max_query_length
# Truncate context
truncated_context = tokenizer.decode(
    tokenizer(context, max_length=max_context_length, truncation=True)["input_ids"],
    skip_special_tokens=True
)

# Build full prompt with truncated context
full_prompt = f"""Answer the following question based on the provided context.

Context:
{truncated_context}

Question:
{query}

Answer:"""

# Tokenize and truncate the full prompt
tokenized_output = tokenizer(
    full_prompt,
    return_tensors="pt",
    max_length=max_input_length,
    truncation=True
)

# Decode truncated input
truncated_input = tokenizer.decode(tokenized_output["input_ids"][0], skip_special_tokens=True)
print("\nTruncated Input:")
print(truncated_input)

# Generate response using the truncated input
response = text_generator(truncated_input)

# Store the model's response in the chat history
chat_state.add_to_history_as_model(response[0]['generated_text'])

# Output the result
print("\nAnswer:\n", response[0]['generated_text'])

NameError: name 'chat_state' is not defined

In [2]:
#This class tries to combine the 2 main functions above: The combined class handles both external retrieval and turn-based chat management.
class ChatState:
    def __init__(self, retriever, tokenizer, text_generator, max_input_length=512, max_query_length=128, system=""):
        """
        Initialize the ChatState object.
        
        Args:
            retriever: Vector store retriever for fetching relevant documents.
            tokenizer: Tokenizer for the language model.
            text_generator: Text generation pipeline.
            max_input_length (int): Maximum input length for the model.
            max_query_length (int): Reserved space for the query.
            system (str): System instructions or bot description.
        """
        self.retriever = retriever
        self.tokenizer = tokenizer
        self.text_generator = text_generator
        self.max_input_length = max_input_length
        self.max_query_length = max_query_length
        self.system = system
        self.history = []

    def add_to_history_as_user(self, message):
        """Add a user message to the chat history."""
        self.history.append(f"user\n{message}\n")

    def add_to_history_as_model(self, message):
        """Add a model message to the chat history."""
        self.history.append(f"model\n{message}\n")

    def get_full_prompt(self):
        """Combine chat history into a single string."""
        return "".join([*self.history])

    def send_message(self, query):
        """
        Process a user query, generate a response, and update the chat history.
        
        Args:
            query (str): The user's question.
        
        Returns:
            str: The generated response.
        """
        # Add user query to history
        self.add_to_history_as_user(query)

        # Retrieve relevant documents
        retrieved_docs = self.retriever.get_relevant_documents(query)
        print("Retrieved Documents:")
        for i, doc in enumerate(retrieved_docs):
            print(f"Document {i + 1}: {doc.page_content[:200]}...")
        
        context = "\n".join([doc.page_content for doc in retrieved_docs])

        # Define token limits
        max_context_length = self.max_input_length - self.max_query_length

        # Truncate context
        truncated_context = self.tokenizer.decode(
            self.tokenizer(context, max_length=max_context_length, truncation=True)["input_ids"],
            skip_special_tokens=True
        )

        # Build full prompt with truncated context
        full_prompt = f"""Answer the following question based on the provided context.

Context:
{truncated_context}

Chat History:
{self.get_full_prompt()}

Question:
{query}

Answer:"""

        # Tokenize and truncate the full prompt
        tokenized_output = self.tokenizer(
            full_prompt,
            return_tensors="pt",
            max_length=self.max_input_length,
            truncation=True
        )

        # Decode truncated input
        truncated_input = self.tokenizer.decode(tokenized_output["input_ids"][0], skip_special_tokens=True)
        print("\nTruncated Input:")
        print(truncated_input)

        # Generate response using the truncated input
        response = self.text_generator(truncated_input)

        # Store the model's response in the chat history
        self.add_to_history_as_model(response[0]['generated_text'])

        # Return the generated response
        return response[0]['generated_text']

In [98]:
chat_state = ChatState(retriever, tokenizer, text_generator)
response = chat_state.send_message("Where is KMC located?")
print("\nAnswer:\n", response)

Retrieved Documents:
Document 1: age, ability or income level. Multnomah County Child Care Centers Listing Compiled by the Early Learning Division, Oregon Dept of Education Multnomah County Child Care Family Home Listing Compiled by ...
Document 2: International Students ADVISING SERVICES IKARL MILLER CENTER & State FRONT DESK SERVICES For general information and quick questions, visit our front desk services. A staff or student member can chat ...
Document 3: needs Oregon Early Learning Division Oregon Early Learning Division - Request Complaint and Compliance History - Child Care Providers CONTACT PSU 1825 SW Broadway Portland, OR 97201 Phone: 503-725-300...

Truncated Input:
Answer the following question based on the provided context. Context: age, ability or income level. Multnomah County Child Care Centers Listing Compiled by the Early Learning Division, Oregon Dept of Education Multnomah County Child Care Family Home Listing Compiled by the Early Learning Division, Oregon Dept of

In [93]:
#to maintain state across turns
query = "what is the meaning of kmc?"
response = chat_state.send_message(query)
print("\nAnswer:\n", response)
#print("\nChat History:\n", chat_state.get_history())


ValueError: Argument 'prompts' is expected to be of type List[str], received argument of type <class 'str'>.

In [ ]:
#using chat state alongside retriever

# Initialize the ChatState with your model
chat_state = ChatState(model=llm, system="You are a helpful assistant.")

# User's input
user_message = "What are the latest advancements in AI?"

# Add to history and create prompt
chat_state.add_to_history_as_user(user_message)

# Get the full conversation history
history = chat_state.get_full_prompt()

# Use the retriever to get relevant context
retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 5, "score_threshold": 0.5})

# Retrieve documents for context
context_docs = retriever.retrieve(user_message)  # You can also pass in `history` as context here

# Now pass the history and context to the model prompt
response = llm.generate(history + "\n" + context_docs)

# Add response to history
chat_state.add_to_history_as_model(response)

# Output response
print("Model response:", response)


In [ ]:
# add interface for ui